# H₂ Molecule Homework Assignment
### Quantum Software Development Journey: From Theory to Application with Classiq - Part 3

- Similarly to what we have done in class, in this exercise we will implement the VQE on H2 molecule.
- This time instead of using the built-in methods and functions (such as `Molecule` and `MoleculeProblem`) to difne and solve the problem, you will be provided with a two qubits Hamiltonian.

## Submission
- Submit the completed Jupyter notebook and report via GitHub. Ensure all files are correctly named and organized.
- Use the Typeform link provided in the submission folder to confirm your submission.

## Additional Resources
- [Classiq Documentation](https://docs.classiq.io/latest/)
- The notebook from live session #3

## Important Dates
- **Assignment Release:** 22.5.2024
- **Submission Deadline:** 3.6.2024 (7 A.M GMT+3)

---

Happy coding and good luck!

### Part 1

Given the following Hamiltonian:

$$
\hat{H} = -1.0523 \cdot (I \otimes I) + 0.3979 \cdot (I \otimes Z) - 0.3979 \cdot (Z \otimes I) - 0.0112 \cdot (Z \otimes Z) + 0.1809 \cdot (X \otimes X)
$$

Complete the following code

In [2]:
from typing import List
from classiq import *

HAMILTONIAN = QConstant("HAMILTONIAN", List[PauliTerm], 
                        [
                            PauliTerm([Pauli.I, Pauli.I], -1.0523),
                            PauliTerm([Pauli.I, Pauli.Z], 0.3979),
                            PauliTerm([Pauli.Z, Pauli.I], -0.3979),
                            PauliTerm([Pauli.Z, Pauli.Z], -0.0112),
                            PauliTerm([Pauli.X, Pauli.X], 0.1809),]) #TODO: Complete Hamiltonian

In [12]:
@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    # TODO: Create an ansatz which allows each qubit to have arbitrary rotation
    allocate(2, q)
    U(angles[0], angles[1], angles[2], 0, q[0]) 
    U(angles[0], angles[1], angles[2], 0, q[1])
    #U(angles[0], angles[1], angles[2], 0, q[2])

@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN, # TODO: complete the missing argument
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

qmod =create_model (main, classical_execution_function=cmain)#TODO: complete the line, use classical_execution_function
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/cdeb3942-86af-4118-9adc-42ca3414dc37?version=0.41.2


In [13]:
execution = execute(qprog)
res = execution.result()
# execution.open_in_ide()
vqe_result = res[0].value

In [14]:
print(f"Optimal energy: {vqe_result.energy}")

print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.063741015625
Optimal parameters: {'angles_0': 2.7905108249302994, 'angles_1': 1.628892707107819, 'angles_2': -0.35058382992185155}
Eigenstate: {'00': (0.02209708691207961+0j), '10': (0.17258376589355096+0j), '01': (0.1711632992203644+0j), '11': (0.9697575405739313+0j)}


Does it similar to the `optimal energy` we calculated in class? \ No
Does it similar to the `total energy` we calculated in class? Different too.

### Part 2

**Now, we want to have a more interesting ansatz in our `main`.**  
Add **one** line of code to the `main` function you created in Part 1 that creates **entanglement** between the two qubits.  
Which gate should you use?

In [15]:
@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    # TODO: Create an ansatz which allows each qubit to have arbitrary rotation
    allocate(2, q)
    U(angles[0], angles[1], angles[2], 0, q[0]) 
    U(angles[0], angles[1], angles[2], 0, q[1])
    CX(q[0],q[1])


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN, # TODO: complete the missing argument
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

qmod = create_model (main, classical_execution_function=cmain)#TODO: complete the line, use classical_execution_function
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/3f71fe36-ce7b-409f-be1b-cfaeabb43b47?version=0.41.2


In [17]:
execution = execute(qprog)
res = execution.result()
# execution.open_in_ide()
vqe_result = res[0].value#TODO: complete the line

In [18]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.8483884765625
Optimal parameters: {'angles_0': -2.9967764181949375, 'angles_1': 5.573021041013109, 'angles_2': -3.0558759903455814}
Eigenstate: {'11': (0.05846339666834283+0j), '10': (0.05412658773652741+0j), '01': (0.9968211192335363+0j)}


Does it similar to the `optimal energy` we calculated in class? \ Yes, they are close
Does it similar to the `total energy` we calculated in class? \ No, not at all
What can we learn about the provided from this result Hamiltonian? It would seem that the local simulators do not operate the VQE as effectively as the Classiq simulator engine or the in-built methods